# Imports

In [1]:
import warnings
from typing import Optional, Callable

import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import ArrayLike
from matplotlib.animation import FuncAnimation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.utils import resample
from sklearn.neural_network import MLPRegressor
from sklearn.base import RegressorMixin


warnings.filterwarnings("ignore")

# Funcs

## linear regression

In [2]:
def create_linear_regression_training_gif(
    X: ArrayLike,
    y: ArrayLike,
    model: RegressorMixin,
    ideal_func: Optional[Callable] = None,
    ideal_formula: Optional[str] = None,
    filename: str = 'training.gif',
    n_epochs: int = 50
) -> None:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
    
    losses = []
    train_errors = []
    test_errors = []
    coef_history = []
    intercept_history = []
    
    for _ in range(n_epochs):
        model.fit(X_train, y_train)
        
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        train_error = mean_squared_error(y_train, y_train_pred)
        test_error = mean_squared_error(y_test, y_test_pred)
        losses.append(train_error)
        train_errors.append(train_error)
        test_errors.append(test_error)
        
        coef_history.append(model.coef_[0])
        intercept_history.append(model.intercept_)
    
    coefs = np.array(coef_history)
    intercepts = np.array(intercept_history)
    
    coef_min, coef_max = coefs.min(), coefs.max()
    intercept_min, intercept_max = intercepts.min(), intercepts.max()
    
    coef_range = coef_max - coef_min
    intercept_range = intercept_max - intercept_min
    coef_min -= coef_range * 0.1 if coef_range != 0 else 0.1
    coef_max += coef_range * 0.1 if coef_range != 0 else 0.1
    intercept_min -= intercept_range * 0.1 if intercept_range != 0 else 0.1
    intercept_max += intercept_range * 0.1 if intercept_range != 0 else 0.1
    
    coef_grid = np.linspace(coef_min, coef_max, 50)
    intercept_grid = np.linspace(intercept_min, intercept_max, 50)
    C, I = np.meshgrid(coef_grid, intercept_grid)
    
    X_train_flat = X_train.flatten()
    y_train_flat = y_train.flatten()
    diff = y_train_flat[:, np.newaxis, np.newaxis] - (X_train_flat[:, np.newaxis, np.newaxis] * C + I)
    mse_values = np.mean(diff ** 2, axis=0)
    Z = mse_values
    
    fig = plt.figure(figsize=(18, 14))
    gs = fig.add_gridspec(2, 1, height_ratios=[2, 1])
    ax1 = fig.add_subplot(gs[0])
    gs_sub = gs[1].subgridspec(1, 2)
    ax2 = fig.add_subplot(gs_sub[0])
    ax3 = fig.add_subplot(gs_sub[1])
    
    scat_train = ax1.scatter(X_train, y_train, c='blue', label='Train')
    scat_test = ax1.scatter(X_test, y_test, c='red', label='Test')
    line_model, = ax1.plot([], [], 'k-', lw=2, label='ML Model', c='orange')
    
    # Текстовая метка для формул
    formula_text = ax1.text(
        0.05, 0.95, '',
        transform=ax1.transAxes,
        fontsize=12,
        verticalalignment='top',
        bbox=dict(facecolor='white', alpha=0.5)
    )
    
    if ideal_func is not None:
        X_ideal = np.linspace(X.min(), X.max(), 100)
        y_ideal = ideal_func(X_ideal)
        line_ideal, = ax1.plot(X_ideal, y_ideal, 'g--', lw=2, label='Ideal Model')
        if ideal_formula:
            formula_text.set_text(f'Real: y = {ideal_formula}')
    
    ax1.set_title('Data and Model')
    ax1.set_xlim(X.min() - 2, X.max() + 2)
    ax1.set_ylim(y.min() - 2, y.max() + 2)
    ax1.legend()
    ax1.grid(True)
    
    loss_line, = ax2.plot([], [], 'b-', label='Training Loss')
    ax2.set_title('Training Loss Over Epochs')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.set_xlim(0, n_epochs)
    ax2.set_ylim(0, max(losses) + 0.1)
    ax2.legend()
    
    table_text = ax2.text(
        0.6, 0.9, '',
        transform=ax2.transAxes,
        fontsize=12,
        verticalalignment='top',
        bbox=dict(facecolor='white', alpha=0.5)
    )
    
    contour = ax3.contourf(C, I, Z, levels=20, cmap='viridis', alpha=0.6)
    plt.colorbar(contour, ax=ax3, label='MSE')
    path_line, = ax3.plot([], [], 'r-', lw=2, label='Parameter Path')
    current_point, = ax3.plot([], [], 'ro', markersize=8, label='Current Parameters')
    ax3.set_xlabel('Coefficient')
    ax3.set_ylabel('Intercept')
    ax3.set_title('Gradient Descent Trajectory')
    ax3.set_xlim(coef_min, coef_max)
    ax3.set_ylim(intercept_min, intercept_max)
    ax3.legend()
    
    def _update(frame):
        if frame < len(coef_history):
            coef = coef_history[frame]
            intercept = intercept_history[frame]
            X_plot = np.linspace(X.min(), X.max(), 100)
            y_model = X_plot * coef + intercept
            line_model.set_data(X_plot, y_model)
            
            current_formula = f'Fitted: y = {coef.item():.2f}x + {intercept.item():.2f}'
            if ideal_formula:
                formula_text.set_text(f'Real: y = {ideal_formula[0]}\n{current_formula}')
            else:
                formula_text.set_text(current_formula)
        
        loss_line.set_data(np.arange(frame + 1), losses[:frame + 1])
        
        text = f'Epoch: {frame+1}\nTrain MSE: {train_errors[frame]:.4f}\nTest MSE: {test_errors[frame]:.4f}'
        table_text.set_text(text)
        
        if frame < len(coef_history):
            path_coefs = coef_history[:frame + 1]
            path_intercepts = intercept_history[:frame + 1]
            path_line.set_data(path_coefs, path_intercepts)
            current_point.set_data([coef], [intercept])
        
        return (line_model, loss_line, table_text, path_line, current_point, formula_text)
    
    ani = FuncAnimation(fig, _update, frames=n_epochs, blit=True, interval=200)
    ani.save(filename, writer='imagemagick', fps=5, dpi=65)
    plt.close()

## decision tree

In [3]:
def create_decision_tree_gif(
    X: ArrayLike,
    y: ArrayLike,
    ideal_func: Optional[Callable] = None,
    filename: str = 'tree.gif', 
    max_depth: int = 5,
    fps: int = 2,
    min_samples_leaf: int = 1
) -> None:
    X = X.reshape(-1, 1)
    X_range = np.linspace(X.min()-1, X.max()+1, 100).reshape(-1, 1)
    
    fig = plt.figure(figsize=(18, 16))
    gs = fig.add_gridspec(2, 2)

    ax_data = fig.add_subplot(gs[0, :])
    ax_tree = fig.add_subplot(gs[1, 0])
    ax_error = fig.add_subplot(gs[1, 1])

    ax_data.scatter(X, y, c='blue', alpha=0.6, label='Data')
    pred_line, = ax_data.plot([], [], 'r-', lw=2, label='Prediction')
    
    if ideal_func is not None:
        X_ideal = np.linspace(X.min(), X.max(), 100)
        ax_data.plot(X_ideal, ideal_func(X_ideal), 'g--', label='Ideal')
    
    ax_data.set(xlim=(X.min()-1, X.max()+1),
              ylim=(y.min()-1, y.max()+1),
              title='Decision Tree Predictions')
    ax_data.legend()
    
    ax_error.set(xlabel='Tree Depth', ylabel='MSE',
                title='Error Progression', 
                xlim=(0, max_depth), 
                ylim=(0, np.max(y)*2))
    error_line, = ax_error.plot([], [], 'b-o', label='MSE')
    ax_error.legend()
    ax_error.grid(True)
    
    info_text = ax_data.text(0.05, 0.95, '', 
                            transform=ax_data.transAxes,
                            bbox=dict(facecolor='white', alpha=0.8))
    
    depths = []
    errors = []
    

    def init():
        pred_line.set_data([], [])
        error_line.set_data([], [])
        return pred_line, error_line
    

    def update(depth):
        tree = DecisionTreeRegressor(
            max_depth=depth+1,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
        tree.fit(X, y)
        
        y_pred = tree.predict(X)
        current_error = mean_squared_error(y, y_pred)
        
        depths.append(depth+1)
        errors.append(current_error)
        
        X_plot = np.linspace(X.min()-1, X.max()+1, 100)
        y_plot = tree.predict(X_plot.reshape(-1, 1))
        pred_line.set_data(X_plot, y_plot)
        
        ax_tree.clear()
        plot_tree(tree, ax=ax_tree, filled=True, 
                 feature_names=['X'], rounded=True)
        ax_tree.set_title(f'Decision Tree (Depth {depth+1})')
        
        error_line.set_data(depths, errors)
        ax_error.set_ylim(0, max(errors)*1.1)
        
        info_text.set_text(f'Depth: {depth+1}\nMSE: {current_error:.4f}')
        
        return pred_line, error_line, info_text
    

    ani = FuncAnimation(fig, update, frames=range(max_depth),
                       init_func=init, blit=False, interval=1000//fps)
    
    ani.save(filename, writer='pillow', fps=fps, dpi=65)
    plt.close()

## random forest

In [4]:
def create_forest_training_gif(
    X: ArrayLike,
    y: ArrayLike,
    ideal_func: Optional[Callable] = None,
    filename: str = 'forest_training.gif',
    n_estimators: int = 10,
    max_depth: int = 3,
    fade_steps: int = 40,
    fps: int = 2
) -> None:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42
    )
    X_train = X_train.reshape(-1, 1)
    X_test = X_test.reshape(-1, 1)
    
    estimators = []
    all_tree_lines = []
    train_errors, test_errors = [], []
    X_range = np.linspace(X.min()-1, X.max()+1, 100).reshape(-1, 1)

    fig = plt.figure(figsize=(18, 16))
    gs = fig.add_gridspec(2, 2)
    ax_data = fig.add_subplot(gs[0, :])
    ax_tree = fig.add_subplot(gs[1, 0])
    ax_error = fig.add_subplot(gs[1, 1])

    ax_data.scatter(X_train, y_train, c='blue', label='Train', alpha=0.6)
    ax_data.scatter(X_test, y_test, c='red', label='Test', alpha=0.6)
    avg_line, = ax_data.plot([], [], 'orange', linewidth=3, label='Average Prediction', zorder=999)
    phantom_line, = ax_data.plot([], [], 'purple', linewidth=2, alpha=0.0)
    
    if ideal_func:
        X_ideal = np.linspace(X.min(), X.max(), 100)
        ax_data.plot(X_ideal, ideal_func(X_ideal), 'g--', label='Ideal Function')

    ax_data.set(xlim=(X.min()-1, X.max()+1), ylim=(y.min()-1, y.max()+1),
               title='Data and Predictions')
    ax_data.legend()

    ax_error.set(xlabel='Number of Trees', ylabel='MSE', title='Error Progression',
                xlim=(0, n_estimators), ylim=(0, 10))
    line_train_err, = ax_error.plot([], [], 'b-', label='Train Error')
    line_test_err, = ax_error.plot([], [], 'r-', label='Test Error')
    ax_error.legend()
    ax_error.grid(True)

    info_text = ax_data.text(0.05, 0.95, '', transform=ax_data.transAxes,
                            bbox=dict(facecolor='white', alpha=0.8))


    def init():
        avg_line.set_data([], [])
        return avg_line,


    def update(frame):
        nonlocal estimators, all_tree_lines
        
        if frame < n_estimators:
            bootstrap_indices = resample(range(len(X_train)), random_state=frame)
            X_bs, y_bs = X_train[bootstrap_indices], y_train[bootstrap_indices]
            
            tree = DecisionTreeRegressor(max_depth=max_depth, random_state=frame)
            tree.fit(X_bs, y_bs)
            estimators.append(tree)
            
            ax_tree.clear()
            plot_tree(tree, ax=ax_tree, filled=True, feature_names=['X'], rounded=True)
            ax_tree.set_title(f'Tree {frame+1}/{n_estimators}')
            
            tree_pred = tree.predict(X_range)
            tree_line, = ax_data.plot(X_range.ravel(), tree_pred, color='black', 
                                     alpha=0.22, linewidth=0.5)
            all_tree_lines.append(tree_line)
            
            info_text.set_text(f'Trees: {frame+1}')

        elif frame < n_estimators + fade_steps:
            fade_progress = (frame - n_estimators + 1) / fade_steps
            new_alpha = 0.3 * (1 - fade_progress)
            
            for line in all_tree_lines:
                line.set_alpha(new_alpha)
            
            info_text.set_text(f'Trees: {n_estimators}\n')

        if len(estimators) > 0:
            y_avg = np.mean([est.predict(X_range) for est in estimators], axis=0)
            avg_line.set_data(X_range.ravel(), y_avg)
            
            y_train_pred = np.mean([est.predict(X_train) for est in estimators], axis=0)
            y_test_pred = np.mean([est.predict(X_test) for est in estimators], axis=0)
            train_errors.append(mean_squared_error(y_train, y_train_pred))
            test_errors.append(mean_squared_error(y_test, y_test_pred))
            
            line_train_err.set_data(np.arange(len(train_errors)), train_errors)
            line_test_err.set_data(np.arange(len(test_errors)), test_errors)
            ax_error.set_ylim(0, max(max(train_errors), max(test_errors)) * 1.1)

        return avg_line, *all_tree_lines, line_train_err, line_test_err, info_text


    total_frames = n_estimators + fade_steps
    ani = FuncAnimation(fig, update, frames=total_frames, init_func=init, 
                       blit=False, interval=1000//fps)
    
    ani.save(filename, writer='pillow', fps=fps, dpi=65)
    plt.close()

## gradient busting

In [5]:
def create_boosting_training_gif(
    X: ArrayLike,
    y: ArrayLike,
    ideal_func: Optional[Callable] = None,
    filename: str = 'boosting.gif',
    n_estimators: int = 20,
    learning_rate: float = 0.1,
    max_depth: int = 3,
    fade_steps: int = 15,
    fps: int = 5
) -> None:
    X = X.reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    current_pred = np.full_like(y_train, np.mean(y_train))
    test_pred = np.full_like(y_test, np.mean(y_train))
    residuals = y_train - current_pred
    trees = []
    
    train_errors = [mean_squared_error(y_train, current_pred)]
    test_errors = [mean_squared_error(y_test, test_pred)]

    fig = plt.figure(figsize=(18, 16))
    gs = fig.add_gridspec(2, 2)
    ax_main = fig.add_subplot(gs[0, :])
    ax_tree = fig.add_subplot(gs[1, 0])
    ax_error = fig.add_subplot(gs[1, 1])

    ax_main.scatter(X_train, y_train, c='blue', alpha=0.6, label='Train')
    ax_main.scatter(X_test, y_test, c='red', alpha=0.6, label='Test')
    main_line, = ax_main.plot([], [], 'orange', lw=2, label='Current Prediction')
    correction_line, = ax_main.plot([], [], 'purple', ls='--', alpha=0.7, label='Correction')
    
    if ideal_func:
        X_ideal = np.linspace(X.min(), X.max(), 100)
        ax_main.plot(X_ideal, ideal_func(X_ideal), 'g--', label='Ideal')
    
    ax_main.set(xlim=(X.min()-1, X.max()+1), 
              ylim=(y.min()-1.5, y.max()+1.5),
              title='Gradient Boosting Process')
    ax_main.legend()
    
    ax_error.set(xlabel='Iteration', ylabel='MSE', 
                title='Error Progression', xlim=(0, n_estimators))
    train_line, = ax_error.plot([], [], 'b-', label='Train')
    test_line, = ax_error.plot([], [], 'r-', label='Test')
    ax_error.legend()
    ax_error.grid(True)
    
    info_text = ax_main.text(0.05, 0.95, '', transform=ax_main.transAxes,
                            bbox=dict(facecolor='white', alpha=0.8))

    X_range = np.linspace(X.min()-1, X.max()+1, 100).reshape(-1, 1)
    full_pred = np.full((100,), np.mean(y_train))


    def init():
        main_line.set_data([], [])
        correction_line.set_data([], [])
        return main_line, correction_line


    def update(frame):
        nonlocal current_pred, test_pred, residuals, full_pred, trees
        
        if frame < n_estimators:
            tree = DecisionTreeRegressor(max_depth=max_depth, random_state=frame)
            tree.fit(X_train, residuals)
            trees.append(tree)
            
            train_correction = learning_rate * tree.predict(X_train)
            test_correction = learning_rate * tree.predict(X_test)
            range_correction = learning_rate * tree.predict(X_range)
            
            current_pred += train_correction
            test_pred += test_correction
            full_pred += range_correction
            
            residuals = y_train - current_pred

            ax_tree.clear()
            plot_tree(tree, ax=ax_tree, filled=True, feature_names=['X'], rounded=True)
            ax_tree.set_title(f'Correction Tree {frame+1}')
            
            train_errors.append(mean_squared_error(y_train, current_pred))
            test_errors.append(mean_squared_error(y_test, test_pred))
        
        main_line.set_data(X_range.ravel(), full_pred)
        train_line.set_data(np.arange(len(train_errors)), train_errors)
        test_line.set_data(np.arange(len(test_errors)), test_errors)
        ax_error.set_ylim(0, max(max(train_errors), max(test_errors)) * 1.1)
        
        info = f'Iteration: {min(frame+1, n_estimators)}\nLearning Rate: {learning_rate}\nResiduals Mean: {residuals.mean():.2f}'
        info_text.set_text(info)
        
        return main_line, correction_line, train_line, test_line


    total_frames = n_estimators + fade_steps
    ani = FuncAnimation(fig, update, frames=total_frames, init_func=init, 
                       blit=False, interval=1000//fps)
    
    ani.save(filename, writer='pillow', fps=fps, dpi=65)
    plt.close()

## neural network

In [6]:
def create_nn_training_gif(
    X: ArrayLike,
    y: ArrayLike,
    ideal_func: Optional[Callable] = None,
    filename: str = 'nn_training.gif',
    hidden_layer_sizes: tuple[int] = (10,),
    max_epochs: int = 500,
    display_epoch_step: int = 20,
    fps: int = 10,
    lr: float = 0.01
) -> None:
    X = X.reshape(-1, 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    X_range = np.linspace(X.min()-1, X.max()+1, 100).reshape(-1, 1)

    model = MLPRegressor(hidden_layer_sizes=hidden_layer_sizes, 
                        max_iter=1, 
                        warm_start=True,
                        learning_rate_init=lr,
                        random_state=42)
    
    model.partial_fit(X_train, y_train)

    predictions = []
    train_loss = []
    test_loss = []
    epochs = []

    fig = plt.figure(figsize=(18, 16))
    gs = fig.add_gridspec(2, 2)
    ax_main = fig.add_subplot(gs[0, :])
    ax_loss = fig.add_subplot(gs[1, 0])
    ax_weights = fig.add_subplot(gs[1, 1])

    ax_main.scatter(X_train, y_train, c='blue', alpha=0.6, label='Train')
    ax_main.scatter(X_test, y_test, c='red', alpha=0.6, label='Test')
    pred_line, = ax_main.plot([], [], 'orange', lw=2, label='Prediction')
    
    if ideal_func:
        X_ideal = np.linspace(X.min(), X.max(), 100)
        ax_main.plot(X_ideal, ideal_func(X_ideal), 'g--', label='Ideal')

    ax_main.set(xlim=(X.min()-1, X.max()+1),
              ylim=(y.min()-1.5, y.max()+1.5),
              title='Neural Network Learning Process')
    ax_main.legend()

    ax_loss.set(xlabel='Epoch', ylabel='MSE',
               title='Loss Progression', 
               xlim=(0, max_epochs), 
               ylim=(0, np.max(y)*2))
    loss_line_train, = ax_loss.plot([], [], 'b-', label='Train Loss')
    loss_line_test, = ax_loss.plot([], [], 'r-', label='Test Loss')
    ax_loss.legend()
    ax_loss.grid(True)

    ax_weights.set_title('Weight Distribution')
    ax_weights.set_xlabel('Weight Value')
    ax_weights.set_ylabel('Frequency')

    info_text = ax_main.text(0.05, 0.95, '', transform=ax_main.transAxes,
                            bbox=dict(facecolor='white', alpha=0.8))


    def init():
        pred_line.set_data([], [])
        loss_line_train.set_data([], [])
        loss_line_test.set_data([], [])
        return pred_line, loss_line_train, loss_line_test


    def update(frame):
        model.partial_fit(X_train, y_train)

        if frame % display_epoch_step == 0:
            current_pred = model.predict(X_range)
            predictions.append(current_pred)
            
            train_pred = model.predict(X_train)
            test_pred = model.predict(X_test)
            train_loss.append(mean_squared_error(y_train, train_pred))
            test_loss.append(mean_squared_error(y_test, test_pred))
            epochs.append(frame)

            weights = []
            for mat in model.coefs_:
                weights.extend(mat.flatten())
            for vec in model.intercepts_:
                weights.extend(vec.flatten())
            
            pred_line.set_data(X_range.ravel(), current_pred)
            loss_line_train.set_data(epochs, train_loss)
            loss_line_test.set_data(epochs, test_loss)
            ax_loss.set_ylim(0, max(max(train_loss), max(test_loss)) * 1.1)
            
            ax_weights.clear()
            ax_weights.hist(weights, bins=30, alpha=0.7, color='purple')
            ax_weights.set_title(f'Weight Distribution (Epoch {frame})')
            ax_weights.set_xlabel('Weight Value')
            ax_weights.set_ylabel('Frequency')
            
            info = f'''Epoch: {frame}
            Train Loss: {train_loss[-1]:.4f}
            Test Loss: {test_loss[-1]:.4f}
            LR: {lr}'''
            info_text.set_text(info)

        return pred_line, loss_line_train, loss_line_test, info_text


    ani = FuncAnimation(fig, update, frames=range(max_epochs),
                       init_func=init, blit=False, interval=1000//fps)

    ani.save(filename, writer='pillow', fps=fps, dpi=65)
    plt.close()

# setup

## linear

In [7]:
k = -0.1
b = 3

linear_func_formula = f"{k}x + {b}",

def linear_func(x):
    return k * x + b

n = 250

np.random.seed(0)
X_linear = np.linspace(-20, 20, n)
y_linear = linear_func(X_linear) + 1 * np.random.randn(n)

## non linear

In [8]:
def non_linear_func(x):
    return np.sin(np.abs(x * 0.33)) * 3

non_linear_func_formula = "y = 3sin(∣0.33x∣)"


n = 500

np.random.seed(0)
X_non_linear = np.linspace(-30, 30, n)
y_non_linear = non_linear_func(X_non_linear) + 1 * np.random.randn(n)

# Linear regression

In [9]:
model_params = dict(
    max_iter=1,
    warm_start=True,
    eta0=0.0023
)

## linear dist

In [10]:
model = SGDRegressor(**model_params)

create_linear_regression_training_gif(
    X_linear.reshape(-1, 1),
    y_linear,
    model,
    ideal_func=linear_func,
    ideal_formula=linear_func_formula,
    filename='linear_model_linear_data_training.gif',
    n_epochs=25
)

MovieWriter imagemagick unavailable; using Pillow instead.


## non linear dist

In [11]:
model = SGDRegressor(**model_params)

create_linear_regression_training_gif(
    X_non_linear.reshape(-1, 1),
    y_non_linear,
    model,
    ideal_func=non_linear_func,
    ideal_formula=non_linear_func_formula,
    filename='linear_model_non_linear_data_training.gif',
    n_epochs=25
)

MovieWriter imagemagick unavailable; using Pillow instead.


# Decision Tree

## non linear dist

In [12]:
create_decision_tree_gif(
    X_non_linear,
    y_non_linear, 
    ideal_func=non_linear_func, 
    filename='tree_training.gif', 
    max_depth=10,
    fps=1,
    min_samples_leaf=1
)

# Random Forest

## non linear dist

In [13]:
create_forest_training_gif(
    X_non_linear,
    y_non_linear, 
    ideal_func=non_linear_func, 
    filename='forest_training.gif', 
    n_estimators=100,
    max_depth=10,
    fps=10
)

# GB

## non linear dist

In [14]:
create_boosting_training_gif(
    X_non_linear, y_non_linear,
    ideal_func=non_linear_func,
    filename='boosting_learning.gif',
    n_estimators=100,
    learning_rate=0.14,
    max_depth=2,
    fade_steps=48,
    fps=24
)

# NN

## non linear dist

In [15]:
create_nn_training_gif(
    X_non_linear, y_non_linear,
    ideal_func=non_linear_func,
    filename='nn_learning.gif',
    hidden_layer_sizes=(50, 50),
    max_epochs=500,
    display_epoch_step=1,
    fps=24,
    lr=0.005
)